In [1]:
""""
Aug 18st, 2020
convert to gzip input and output

# February 12st, 2020
# Renato Kogeyama

# Script to calculate self citations (assignee level)
# the script find the assignee for patent_id and citation_id
# then compare to verify if they are the same
# it probably can be rewritten to improve performance

# produces a citation level dataset with citation identifier plus a dummy indicating self citation
"""

'"\nAug 18st, 2020\nconvert to gzip input and output\n\n# February 12st, 2020\n# Renato Kogeyama\n\n# Script to calculate self citations (assignee level)\n# the script find the assignee for patent_id and citation_id\n# then compare to verify if they are the same\n# it probably can be rewritten to improve performance\n\n# produces a citation level dataset with citation identifier plus a dummy indicating self citation\n'

In [2]:
import pandas as pd
import numpy as np
# import itertools
# import seaborn as sns
# import networkx as nx
# import csv
# from graphviz import Digraph
# import igraph
# import matplotlib.pyplot as plt
# import datetime


# import sys
# sys.path.append('/home/rkogeyam/scripts/')
# sys.path.append('scripts/')

# from determinants_scripts import classes, dtypes
import gzip

In [3]:
asg='data/patent_assignee.tsv.gz'
cit='data/cleanuspatentcitation.csv.gz'

dst='data/self_cit.csv.gz'

# names=['index', 'id', 'cit_received']
# usecols=['id', 'cit_received']
# dtype={'main_group':'int64', 'subgroup':'int64'}
# cols=['patent_id','citation_id', 'cit_delay']

usecols=['patent_id', 'assignee_id']
file_asg=gzip.open(asg, 'r')
df_asg=pd.read_csv(file_asg, sep='\t', usecols=usecols)

usecols=['uuid','patent_id', 'citation_id']
file_cit=gzip.open(cit, 'r')
df_cit=pd.read_csv(file_cit, usecols=usecols)

# df=pd.read_csv(src, chunksize=10000)

# df=pd.read_csv(src, dtype=dtypes, usecols=usecols, parse_dates=['date'], index_col='id', chunksize=10000)
# df = pd.read_csv(src, chunksize=1000, usecols=usecols, dtypes=dtypes, parse_dates=['date'])

# df = pd.read_csv(src, chunksize=1000,  parse_dates=['date'], error_bad_lines=False)
# df = pd.read_csv(src, usecols=usecols, names=names, chunksize=1000, dtype=dtype, index_col='id')
# df=pd.read_csv(src, usecols=[0, 1, 3], dtype={0:object}, index_col=0, chunksize=1000)

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
%matplotlib inline

In [5]:
df_asg['patent_id']=df_asg['patent_id'].astype(str)
df_asg.info

patent_id      object
assignee_id    object
dtype: object

In [6]:
df_asg.set_index('patent_id', inplace=True)

In [7]:
df_cit['patent_id']=df_cit['patent_id'].astype(str)
df_cit.info()

uuid           object
patent_id      object
citation_id    object
dtype: object

In [8]:
df_cit.set_index('patent_id', inplace=True)

In [9]:
df_cit=df_cit.join(df_asg)
df_cit.head()

,uuid,citation_id,assignee_id
patent_id,,,
10000000,eyqqavzvg46oies1s1jajx45o,5093563,org_xpEol9Bqufk9r7H1oQhs
10000000,sk41vanuv913q2oq1gu16kelk,5751830,org_xpEol9Bqufk9r7H1oQhs
10000001,0iuh6sxpfunlh8pm9fkkycz97,9296144,org_mHLTB0b0i1bxDE6MnWFP
10000001,hubkpzxws50xbyfft6ioi60b8,7804268,org_mHLTB0b0i1bxDE6MnWFP
10000001,nfkhcwvzleqs4u97rrv19kqf1,9566732,org_mHLTB0b0i1bxDE6MnWFP


In [ ]:
df_cit.info()

In [10]:
df_cit.rename(columns={'assignee_id': 'pat_asg'}, inplace=True)

In [11]:
df_cit.reset_index(inplace=True)

In [12]:
df_cit=df_cit.set_index('citation_id')

In [13]:
%%time
df_cit=df_cit.join(df_asg)

CPU times: user 4min 37s, sys: 5.24 s, total: 4min 42s
Wall time: 4min 42s


In [ ]:
df_cit.info()

In [14]:
df_cit.head()

,patent_id,uuid,pat_asg,assignee_id
0,5351803,eeggwd3pnsfcara1i60o42hdu,org_C7LdmoC8j4mUth9Rko4c,NaN
0000000,5489070,xjdvdj2y0lgdas51rewjzn405,org_Fj2cKELVyvFNgZBuxxcB,NaN
000004,7017481,ifomzwo6g13prnj5c29mgj8k4,org_EkYeo6ADBTltFZiksTcd,NaN
000006,6813147,a6crepaxejht8e16czfm0kytd,org_XcqCjtT8pi8Mk3UjQlDt,NaN
0000482,5054020,olkyv8qnteatjed767vodhmlz,org_xmHc3Az9MhLSDD4X9xBu,NaN


In [15]:
df_cit.rename(columns={'assignee_id':'cit_asg'}, inplace=True)

df_cit['self_cit']=np.where(df_cit['cit_asg'] == df_cit['pat_asg'], 1, 0)

In [16]:
df_cit.head()

,patent_id,uuid,pat_asg,cit_asg,self_cit
0,5351803,eeggwd3pnsfcara1i60o42hdu,org_C7LdmoC8j4mUth9Rko4c,NaN,0
0000000,5489070,xjdvdj2y0lgdas51rewjzn405,org_Fj2cKELVyvFNgZBuxxcB,NaN,0
000004,7017481,ifomzwo6g13prnj5c29mgj8k4,org_EkYeo6ADBTltFZiksTcd,NaN,0
000006,6813147,a6crepaxejht8e16czfm0kytd,org_XcqCjtT8pi8Mk3UjQlDt,NaN,0
0000482,5054020,olkyv8qnteatjed767vodhmlz,org_xmHc3Az9MhLSDD4X9xBu,NaN,0


In [17]:
df_cit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113156459 entries, 0 to nan
Data columns (total 5 columns):
patent_id    object
uuid         object
pat_asg      object
cit_asg      object
self_cit     int64
dtypes: int64(1), object(4)
memory usage: 5.1+ GB


In [18]:
# df_cit[['patent_id', 'self_cit']].to_csv(dst, index_label='citation_id')
df_cit[['uuid','self_cit']].to_csv(dst, index=False, compression='gzip')